<a href="https://colab.research.google.com/github/It-HyunJin/Graduation/blob/main/book_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [2]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.0 MB/s eta 0:00:00


In [22]:
!pip install webdriver-manager

In [71]:
!pip install beautifulsoup4

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [102]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import csv
import os
import json

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

all_reviews = load_progress()
completed_book_links = load_progress()

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=chrome_options)
time.sleep(3)

In [48]:
driver.implicitly_wait(10)

In [49]:
all_reviews = []

In [93]:
category_urls={
    '소설/시/희곡':'https://www.yes24.com/product/category/bestseller?pageNumber=1&pageSize=100&categoryNumber=001001046',
    '에세이': 'https://www.yes24.com/product/category/bestseller?pageNumber=1&pageSize=100&categoryNumber=001001047',
    '자기계발':'https://www.yes24.com/product/category/bestseller?pageNumber=1&pageSize=100&categoryNumber=001001026',
    'IT':'https://www.yes24.com/product/category/bestseller?pageNumber=1&pageSize=100&categoryNumber=001001003',
    '여행':'https://www.yes24.com/product/category/bestseller?pageNumber=1&pageSize=100&categoryNumber=001001009',
    '인문':'https://www.yes24.com/product/category/bestseller?pageNumber=1&pageSize=100&categoryNumber=001001019',
    '건강/취미':'https://www.yes24.com/product/category/bestseller?pageNumber=1&pageSize=100&categoryNumber=001001011'
}

BOOK_PER_CATEGORY = 100
SAVE_CSV_PATH = './yes24_reviews.csv'
PROGRESS_PATH = './yest24_progress.json'

In [ ]:
def save_progress(completed_links):
  with open(PROGRESS_PATH, 'w', encoding='utf-8-sig') as f:
    json.dump(list(completed_links), f, ensure_ascii=False, indent=2)
  print(f"[진생상황 저장 완료]: 완료 책: {len(completed_links)}")

def load_progress():
  if os.path.exists(SAVE_CSV_PATH) and os.path.exists(PROGRESS_PATH):
    with open(SAVE_CSV_PATH, 'r', encoding='utf-8-sig', newline='') as f:
      reader = csv.DictReader(f)
      all_reviews = list(reader)
    with open(PROGRESS_PATH, 'r', encoding='utf-8-sig') as f:
      completed_links = set(json.load(f))
    print(f"[이전 작업 불러오기 완료] 리뷰 {len(all_reviews)}개, 완료한 책 {len(completed_links)}개")
    return all_reviews, completed_links
  else:
    return [], set()

def save_data_to_csv(data):
  fieldnames = ['책이름', '저자', '책ID', '별점', '리뷰']
  file_exists = os.path.exists(SAVE_CSV_PATH)

  with open(SAVE_CSV_PATH, 'a', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    if not file_exists:
      writer.writeheader()
    for row in data:
      writer.writerow(row)

  print(f"[CSV 저장 완료] 총 {len(data)}개 저장")

In [ ]:
print("start crawling")

all_reviews = []

for category_name, category_url_base in category_urls.items(): # 변수명 좀 더 명확하게 변경 (기본 URL)
  print(f"------------- 카테고리: '{category_name}' 수집 시작-------------")
  book_count = 0 # 현재 카테고리에서 수집한 책 권수
  current_page = 1 # 현재 카테고리 페이지 번호

  completed_book_links = set() # 이미 처리한 책 링크 저장 (중복 방지)

  # 목표 권수를 채우거나 더 이상 책이 없을 때까지 반복
  while book_count < BOOK_PER_CATEGORY:
    # 다음 페이지 URL 생성
    category_page_url = f"{category_url_base}"
    print(f"'{category_name}' 카테고리의 {current_page} 페이지로 이동중: {category_page_url}")

    try:
      driver.get(category_page_url) # <-- 페이지 번호가 붙은 URL로 이동
      time.sleep(3) # 페이지 로딩 대기

      # 현재 카테고리 페이지의 HTML 소스 파싱
      soup_category = BeautifulSoup(driver.page_source, 'html.parser') # 변수명 변경

      book_item_links_on_page = soup_category.select('div.item_info > div.info_row.info_name > a.gd_name') # 책 링크를 가진 a 태그 직접 선택

      if not book_item_links_on_page:
        print(f"'{category_name}' 카테고리의 {current_page} 페이지에서 책을 찾을 수 없습니다. ({len(completed_book_links)}권 수집 완료)")
        break # 현재 페이지에 더 이상 책이 없으면 카테고리 루프 종료

      print(f"{len(book_item_links_on_page)}개의 책 링크 찾음")

      # 현재 페이지에서 찾은 책 링크들을 순회
      for book_link_element in book_item_links_on_page:
        if book_count >= BOOK_PER_CATEGORY:
          print(f"'{category_name}' 카테고리에서 목표 {BOOK_PER_CATEGORY}권 달성!")
          break # 목표 권수 달성 시 내부 루프 종료

        book_detail_url = book_link_element['href'] # href 속성에서 상세 페이지 링크 추출
        # 상대 경로 링크일 경우 절대 경로로 변환
        if book_detail_url.startswith('/'):
          book_detail_url = 'https://www.yes24.com' + book_detail_url # 예스24 기본 URL 붙여주기

        # 이미 처리한 책인지 확인
        if book_detail_url in completed_book_links:
          print(f"  이미 처리한 책 링크입니다: {book_detail_url}")
          continue # 이미 처리했으면 건너뛰기

        completed_book_links.add(book_detail_url) # 처리할 책 링크를 set에 추가
        book_count += 1 # 수집할 책 권수 카운트 증가
        print(f"--- ({book_count}/{BOOK_PER_CATEGORY}) 책 상세 페이지 이동: {book_detail_url} ---")

        try:
          # 책 상세 페이지로 이동
          driver.get(book_detail_url)
          # 리뷰 목록 영역이 나타날 때까지 대기
          WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#infoset_reviewContentList'))) # 리뷰 영역 ID (예시, 확인 필요!)
          time.sleep(2)

          # 책 상세 페이지
          book_soup = BeautifulSoup(driver.page_source, 'html.parser')

          # --- 책 기본 정보 (책이름, 저자, 책id, 전체 별점) 추출 ---
          book_title = book_soup.select_one('h2.gd_name').get_text(strip=True) if book_soup.select_one('h2.gd_name') else 'N/A'
          book_author_element = book_soup.select_one('span.gd_auth > a') or book_soup.select_one('span.gd_auth') # 저자 정보 셀렉터 확인 필요, a 태그 없을 수도 있음
          book_author = book_author_element.get_text(strip=True) if book_author_element else 'N/A'
          # 책 ID는 상세 페이지 URL에서 추출
          book_id = book_detail_url.split('/')[-1] if book_detail_url.split('/')[-1].isdigit() else 'N/A'
          # 전체 별점
          overall_rating_element = book_soup.select_one(' em.yes_b')
          overall_rating = overall_rating_element.get_text(strip=True) if overall_rating_element else 'N/A'

          print(f"    책 제목: {book_title}, 저자: {book_author}, ID: {book_id}, 전체 별점: {overall_rating}")

          #리뷰페이지 넘기며 수집
          review_page_num = 1

          book_id_for_reviews = book_detail_url.split('/')[-1]

          while True:
            current_review_page_url=(
                f"https://www.yes24.com/Product/CommunityModules/GoodsReviewList/{book_id_for_reviews}"
                f"?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N,N&goodsGb=01&Sort=2"
                f"&PageNumber={review_page_num}&Type=ALL&DojungAfterBuy=0")

            print(f"    리뷰 {review_page_num} 페이지로 이동: {current_review_page_url}")

            try:
                  driver.get(current_review_page_url)
                  WebDriverWait(driver,10).until(lambda d: len(BeautifulSoup(d.page_source, 'html.parser').select('div.reviewInfoBot.origin div.review_cont')) > 0 or \
                                "해당 책의 리뷰가 없습니다." in d.page_source # 리뷰가 없다는 텍스트가 있는지 함께 확인
                  )

                  time.sleep(2)

                  soup_final_reviews = BeautifulSoup(driver.page_source, 'html.parser')

                  review_element_selector = 'div.reviewInfoBot.origin div.review_cont'
                  review_elements = soup_final_reviews.select(review_element_selector)

                  if not review_elements:
                   print("      해당 책에 리뷰가 없습니다.")
              # 리뷰가 없을 경우 처리 (책 정보는 저장하고 리뷰는 비워두기)
                   all_reviews.append({
                      '책이름': book_title,
                      '저자': book_author,
                      '책id': book_id,
                      '별점': overall_rating,
                      '리뷰': '리뷰 없음'
                 })
                  else:
                   print(f"      총 {len(review_elements)}개의 리뷰 발견.")
                  for review_element in review_elements:

                     review_text_element = review_element.select_one(' .review_cont')

                     review_text = review_element.get_text(strip=True) if review_element else 'N/A'

                  # 수집한 데이터를 리스트에 추가 (각 리뷰마다 한 행)
                     all_reviews.append({
                       '책이름': book_title,
                       '저자': book_author,
                       '책id': book_id,
                       '리뷰': review_text
                   })
                     print(f"        내용: {review_text[:50]}...") # 확인용 일부 출력

                  review_page_num += 1

            except Exception as e:
              print(f"    !!! 리뷰 페이지 {review_page_num} 처리 중 오류 발생: {e}")
              break # 리뷰 페이지 처리 중 오류 발생 시 현재 책의 리뷰 수집 중단

        except Exception as e:
          print(f"    !!! 책 상세 페이지 또는 리뷰 크롤링 중 오류 발생: {e} (URL: {book_detail_url})")
            # 오류 발생 시 해당 책은 건너뛰거나, 오류 정보를 저장하는 등의 처리 필요

        # 다음 책으로 넘어가기 전에 잠시 대기 (서버 부하 방지)
        time.sleep(1)

        if book_count % 10 == 0:
          print("[중간 저장]")
          save_data_to_csv(all_reviews)
          save_progress(completed_book_links)
          all_reviews.clear()

      # 현재 페이지의 책 링크들을 모두 처리했으면 다음 페이지로 이동
      current_page += 1

    except Exception as e:
      print(f"!!! 카테고리 페이지 이동 또는 처리 중 오류 발생: {e} (URL: {category_page_url})")
      break

  print(f"------------- 카테고리: '{category_name}' 수집 완료 ({len([r for r in all_reviews if r['책이름'] != 'N/A'])}권 데이터 수집) -------------") # 대략적인 권수 확인


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
    리뷰 2 페이지로 이동: https://www.yes24.com/Product/CommunityModules/GoodsReviewList/15718330?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N,N&goodsGb=01&Sort=2&PageNumber=2&Type=ALL&DojungAfterBuy=0
      총 5개의 리뷰 발견.
        내용: 우리는 어떤 삶을 살고 있는가<스토너>(알에이치코리아, 2015)존 윌리엄스의<스토너>는1...
        내용: 농부의 아들로 태어난 한 남자의 묵묵한 일생을 그린 소설. 50년 전에 나왔는데 정작 미국...
        내용: 요즘 컨텐츠로서는 빵점이다. 관객들이 사이다가 없으면 절대 보지 않으려고 하는 시대에 <스...
        내용: 살다보면 이런 일도 있고 저런 일도 있지만, 뒤돌아 보면 대개는 자잘한 일들이고, 무엇 하...
        내용: 이렇게 평범한 한 인생이 문학이 되고, 공감을 부르며, 가슴에 남을 수 있다는 게, 애틋하...
    리뷰 3 페이지로 이동: https://www.yes24.com/Product/CommunityModules/GoodsReviewList/15718330?goodsSortNo=001033&resourceKeyGb=01&goodsStateGb=02&goodsSetYn=N,N&goodsGb=01&Sort=2&PageNumber=3&Type=ALL&DojungAfterBuy=0
      총 5개의 리뷰 발견.
        내용: 삶을 적극적으로 개척하는 사람이 있고
그저 삶을 버티는 사람이 있다.좋은 기회나 행운이 찾...
        내용: 우리는 꿈에 대해 질문하기를 좋아한다. 말 그대로 꿈이니까 자유롭게 말할 수 있다고 믿기 ...
        내용: 어쩌면 답답함의 대명사 하면 스토너가 빠지지

In [ ]:
df = pd.DataFrame(all_reviews)
df.to_csv('SAVE_CSV_PATH', index=False, encoding='utf-8-sig')
print(f"[최종저장완료] 총 {len(df)}개 저장: {SAVE_CSV_PATH}")

'''with open(PROGRESS_PATH, 'w', encoding='utf-8-sig') as f:
  json.dump(list(completed_book_links), f, ensure_ascii=False, indent=2)
print(f"[최종저장완료 책 링크]: {PROGRESS_PATH}")'''

In [ ]:
# 수집된 데이터를 Pandas DataFrame으로 만들기
df = pd.DataFrame(all_reviews)

# CSV 파일로 저장 (인코딩 중요!)
csv_filename = 'yes24_book_reviews.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig') # 한글 깨짐 방지

print(f"\n--- 크롤링 완료! 총 {len(df)}개의 리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다. ---")

In [ ]:
# WebDriver 종료
if 'driver' in locals() and driver:
    driver.quit()
    print("WebDriver 종료 완료.")
